In [1]:
#importing libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
import  pathlib
import numpy as np
from collections import Counter
import tensorflow_hub as hub

In [3]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
BATCH_SIZE = 32

In [5]:
#importing dataset 
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "C:/Users/MP/Videos/IMG_CLASSES",
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle=True,
    validation_split = 0.2,
    subset = 'training',
    seed = 123
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "C:/Users/MP/Videos/IMG_CLASSES",
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle=False,
    validation_split=0.2,
    subset='validation',
    seed=123
)

Found 27153 files belonging to 10 classes.
Using 21723 files for training.
Found 27153 files belonging to 10 classes.
Using 5430 files for validation.


In [7]:
#number of classes 
class_names = ds_train.class_names
num_classes = len(class_names)

In [9]:
#data augumentation 

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05),
])

#data augmentation function 

def augment(image, label):
    image = data_augmentation(image, training=True)
    return image, label


In [13]:
#transfer learning using mobileNetV2
IMG_SHAPE = (224, 224, 3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights="imagenet"
)

In [15]:
#data preprocessing

def preprocess(image, label):
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

In [17]:
AUTOTUNE = tf.data.AUTOTUNE
# TRAIN: augmentation + preprocess
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.prefetch(AUTOTUNE)

# VALIDATION: preprocess ONLY (no augmentation)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

In [25]:
# Unfreeze last 20 layers only
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable =False


# Freeze BatchNorm layers
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False


In [27]:
#training the model 

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer((224,224,3)),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    ds_train,
    validation_data=val_ds,
    epochs=10,
)

Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 885s 1s/step - accuracy: 0.4980 - loss: 1.4260 - val_accuracy: 0.4560 - val_loss: 1.5321
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 793s 1s/step - accuracy: 0.6390 - loss: 0.9402 - val_accuracy: 0.5435 - val_loss: 1.1735
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 812s 1s/step - accuracy: 0.6775 - loss: 0.8476 - val_accuracy: 0.6142 - val_loss: 1.0217
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 792s 1s/step - accuracy: 0.7081 - loss: 0.7704 - val_accuracy: 0.5593 - val_loss: 1.1276
Epoch 5/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 715s 1s/step - accuracy: 0.7337 - loss: 0.7081 - val_accuracy: 0.5378 - val_loss: 1.4266
Epoch 6/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 897s 1s/step - accuracy: 0.7415 - loss: 0.6801 - val_accuracy: 0.7055 - val_loss: 0.7746
Epoch 7/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 932s 1s/step - accuracy: 0.7683 - loss: 0.6263 - val_accuracy: 0.5028 - val_loss: 1.5601
Epoch 8/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 833s 1s/step - accuracy: 0.7832 - loss: 0.5825 - val_accu